# PII Text Extractor - Pipeline no Google Colab

Este notebook permite executar o pipeline completo de extração de PII (Dados Pessoais) diretamente no Google Colab.

**Funcionalidades:**
- Clonagem do repositório do GitHub
- Instalação de dependências
- Download do dataset
- Preparação dos dados (chunking)
- Fine-tuning do modelo
- Inferência em textos
- Avaliação do modelo

**Recomendação:** Use um runtime com GPU (T4 ou superior) para treino.

## 1. Configuração do Ambiente

In [ ]:
# @title Configurações {display-mode: "form"}

# URL do repositório no GitHub
GITHUB_REPO = "https://github.com/EliMCosta/pii-text-extractor-pt.git"  # @param {type:"string"}

# Branch a ser clonada
GITHUB_BRANCH = "main"  # @param {type:"string"}

# Diretório de trabalho
WORK_DIR = "/content/pii-text-extractor-pt"

In [ ]:
# Verificar GPU disponível
!nvidia-smi

In [ ]:
# Clonar o repositório do GitHub
import os

if os.path.exists(WORK_DIR):
    print(f"Diretório {WORK_DIR} já existe. Atualizando...")
    %cd {WORK_DIR}
    !git pull origin {GITHUB_BRANCH}
else:
    !git clone --branch {GITHUB_BRANCH} {GITHUB_REPO} {WORK_DIR}
    %cd {WORK_DIR}

print(f"\nDiretório atual: {os.getcwd()}")
!ls -la

In [ ]:
# Instalar dependências
# Nota: Warnings de conflitos com pacotes pré-instalados do Colab podem ser ignorados
%cd {WORK_DIR}

!pip install -q --upgrade -r requirements.txt 2>&1 | grep -v "dependency conflicts"
print("Dependências instaladas com sucesso!")

## 2. Download do Dataset

In [ ]:
# Criar diretório de dados e clonar o dataset do Hugging Face
%cd {WORK_DIR}

!mkdir -p data
%cd data

if os.path.exists("esic-ner"):
    print("Dataset já existe. Atualizando...")
    %cd esic-ner
    !git pull
    %cd ..
else:
    !git clone https://huggingface.co/datasets/EliMC/esic-ner

%cd {WORK_DIR}
print("\nArquivos do dataset:")
!ls -la data/esic-ner/

## 3. Preparação dos Dados (Smart Chunking)

In [ ]:
# Processar o dataset com Smart Chunking
%cd {WORK_DIR}

!python data_preprocessing/build_finetune_jsonl.py \
    --input data/esic-ner/train.jsonl \
    --output data/esic-ner/train_chunks.jsonl \
    --max_length 512 \
    --stride 64

print("\nArquivo processado:")
!ls -lh data/esic-ner/train_chunks.jsonl
!wc -l data/esic-ner/train_chunks.jsonl

## 4. Fine-tuning do Modelo

**Atenção:** Esta etapa pode levar vários minutos dependendo do tamanho do dataset e da GPU disponível.

In [ ]:
# @title Parâmetros de Treinamento {display-mode: "form"}

MODEL_BASE = "neuralmind/bert-base-portuguese-cased"  # @param {type:"string"}
NUM_EPOCHS = 3  # @param {type:"integer"}
BATCH_SIZE = 8  # @param {type:"integer"}
LEARNING_RATE = 5e-5  # @param {type:"number"}
OUTPUT_DIR = "outputs/pii-textx-pt"  # @param {type:"string"}

In [ ]:
# Executar o fine-tuning
%cd {WORK_DIR}

!CUDA_VISIBLE_DEVICES=0 accelerate launch --num_processes 1 \
    training/finetune_pii_token_classification.py \
    --model_name_or_path {MODEL_BASE} \
    --dataset_path data/esic-ner/train_chunks.jsonl \
    --output_dir {OUTPUT_DIR} \
    --num_train_epochs {NUM_EPOCHS} \
    --per_device_train_batch_size {BATCH_SIZE} \
    --learning_rate {LEARNING_RATE} \
    --bf16

In [ ]:
# Verificar o modelo treinado
print("Arquivos do modelo treinado:")
!ls -la {OUTPUT_DIR}/best/

## 5. Inferência

Agora você pode usar o modelo treinado para extrair PII de textos.

In [ ]:
# @title Inferência em Texto {display-mode: "form"}

TEXTO_EXEMPLO = "O CPF do solicitante João Silva é 123.456.789-00. Ele mora na Rua das Flores, 123."  # @param {type:"string"}
MODEL_PATH = "outputs/pii-textx-pt/best"  # @param {type:"string"}

In [ ]:
# Executar inferência via CLI
%cd {WORK_DIR}

!python infer_pii.py \
    --model_name_or_path {MODEL_PATH} \
    infer \
    --text "{TEXTO_EXEMPLO}"

In [ ]:
# Inferência via Python (para integração programática)
import sys
sys.path.insert(0, WORK_DIR)

import json
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
from data_preprocessing.chunking import build_chunks
from inference import (
    get_label_maps_from_model,
    viterbi_decode_bio,
    spans_from_token_predictions_scored,
    filter_scored_spans,
    merge_and_resolve_scored_spans,
)
from ner_labels import PII_TYPES
import numpy as np


def infer_pii(text: str, model_path: str = "outputs/pii-textx-pt/best") -> dict:
    """Executa inferência de PII em um texto."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
    model = AutoModelForTokenClassification.from_pretrained(model_path)
    model.to(device)
    model.eval()
    
    label2id, id2label = get_label_maps_from_model(model)
    o_id = int(label2id["O"])
    
    chunks = build_chunks(
        text=text,
        tokenizer=tokenizer,
        max_length=512,
        stride=64,
        boundary_backoff=32,
    )
    
    token_logits_sum = {}
    token_logits_count = {}
    
    with torch.no_grad():
        for ch in chunks:
            enc = tokenizer(
                ch.text,
                add_special_tokens=True,
                truncation=True,
                max_length=512,
                padding=True,
                return_offsets_mapping=True,
                return_tensors="pt",
            )
            offsets = enc.pop("offset_mapping")[0].tolist()
            attn = enc.get("attention_mask")[0].tolist()
            enc = {k: v.to(device) for k, v in enc.items()}
            logits = model(**enc).logits[0].cpu().numpy()
            
            for ti, (a, b) in enumerate(offsets):
                if a == 0 and b == 0:
                    continue
                if int(attn[ti]) == 0:
                    continue
                ga = int(a) + int(ch.char_start)
                gb = int(b) + int(ch.char_start)
                if ga >= gb:
                    continue
                k = (ga, gb)
                v = logits[ti].astype(np.float32)
                if k in token_logits_sum:
                    token_logits_sum[k] += v
                    token_logits_count[k] += 1
                else:
                    token_logits_sum[k] = v.copy()
                    token_logits_count[k] = 1
    
    if not token_logits_sum:
        return {"text": text, "spans": [], "should_be_public": True}
    
    keys = sorted(token_logits_sum.keys())
    em_global = np.stack(
        [token_logits_sum[k] / token_logits_count[k] for k in keys]
    ).astype(np.float32)
    offs_global = list(keys)
    
    pred_ids = viterbi_decode_bio(
        emissions=em_global,
        id2label=id2label,
        o_id=o_id,
        force_o_mask=None,
    )
    
    spans_scored = spans_from_token_predictions_scored(
        offsets=offs_global,
        pred_ids=pred_ids,
        logits=em_global,
        id2label=id2label,
        o_id=o_id,
        conf_agg="mean",
    )
    spans_scored = filter_scored_spans(spans_scored, conf_threshold=0.0)
    merged = merge_and_resolve_scored_spans(spans_scored, resolve_overlaps=True)
    
    pii_types = set(PII_TYPES)
    should_be_public = not any(s.pii_type in pii_types for s in merged)
    
    return {
        "text": text,
        "spans": [
            {
                "type": s.pii_type,
                "start": s.start,
                "end": s.end,
                "value": text[s.start:s.end],
                "confidence": round(s.confidence, 4),
            }
            for s in merged
        ],
        "should_be_public": should_be_public,
    }


# Testar
resultado = infer_pii(TEXTO_EXEMPLO, MODEL_PATH)
print(json.dumps(resultado, ensure_ascii=False, indent=2))

## 6. Avaliação do Modelo

In [ ]:
# Executar avaliação no dataset de teste
%cd {WORK_DIR}

# Verificar se existe arquivo de teste
import os
test_file = "data/esic-ner/test.jsonl"
if os.path.exists(test_file):
    !python infer_pii.py \
        --model_name_or_path {MODEL_PATH} \
        eval \
        --dataset_path {test_file} \
        --report_path outputs/eval_report.md
else:
    print(f"Arquivo de teste não encontrado: {test_file}")
    print("Usando uma amostra do train.jsonl para demonstração...")
    !head -100 data/esic-ner/train.jsonl > data/esic-ner/sample_test.jsonl
    !python infer_pii.py \
        --model_name_or_path {MODEL_PATH} \
        eval \
        --dataset_path data/esic-ner/sample_test.jsonl \
        --report_path outputs/eval_report.md \
        --max_rows 100

In [ ]:
# Visualizar relatório de avaliação
from IPython.display import Markdown, display

report_path = f"{WORK_DIR}/outputs/eval_report.md"
if os.path.exists(report_path):
    with open(report_path, "r", encoding="utf-8") as f:
        display(Markdown(f.read()))
else:
    print("Relatório não encontrado.")

## 7. Salvar Modelo no Google Drive (Opcional)

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Copiar modelo para o Drive
DRIVE_OUTPUT_PATH = "/content/drive/MyDrive/pii-extractor-model"  # @param {type:"string"}

!mkdir -p "{DRIVE_OUTPUT_PATH}"
!cp -r {WORK_DIR}/{OUTPUT_DIR}/best/* "{DRIVE_OUTPUT_PATH}/"

print(f"Modelo salvo em: {DRIVE_OUTPUT_PATH}")
!ls -la "{DRIVE_OUTPUT_PATH}/"

## 8. Inferência em Lote (JSONL)

In [ ]:
# Criar arquivo de exemplo para inferência em lote
import json

exemplos = [
    {"text": "Meu nome é Maria da Silva e meu CPF é 987.654.321-00."},
    {"text": "O processo SEI 00400-00123456/2024-99 foi instaurado."},
    {"text": "Solicito informações sobre a Lei 12.527/2011."},
    {"text": "Entre em contato pelo e-mail joao.santos@email.com ou telefone (61) 99999-8888."},
]

input_file = f"{WORK_DIR}/data/exemplos_inferencia.jsonl"
with open(input_file, "w", encoding="utf-8") as f:
    for ex in exemplos:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")

print(f"Arquivo criado: {input_file}")
!cat {input_file}

In [ ]:
# Executar inferência em lote
%cd {WORK_DIR}

!python infer_pii.py \
    --model_name_or_path {MODEL_PATH} \
    infer \
    --jsonl_in data/exemplos_inferencia.jsonl \
    --jsonl_out outputs/resultados_inferencia.jsonl

print("\nResultados:")
!cat outputs/resultados_inferencia.jsonl | python -m json.tool --no-ensure-ascii

---

## Usando um Modelo Pré-treinado do Hugging Face Hub

Se você tiver um modelo já publicado no Hugging Face Hub, pode usá-lo diretamente sem precisar treinar.

In [ ]:
# @title Usar Modelo do Hugging Face Hub {display-mode: "form"}

HF_MODEL_ID = "EliMC/pii-text-extractor-pt"  # @param {type:"string"}
TEXTO_TESTE = "O contribuinte José Santos, CPF 111.222.333-44, solicita revisão."  # @param {type:"string"}

In [ ]:
# Inferência com modelo do Hub
%cd {WORK_DIR}

!python infer_pii.py \
    --model_name_or_path {HF_MODEL_ID} \
    infer \
    --text "{TEXTO_TESTE}"